# Importar librerías

In [1]:
import math           # Importa una librería
import pandas as pd   # Importa librería completa y le da un alias
from Bio import SeqIO # Importa una función o clase de una librería

In [2]:
# pip install biopython (instalar desde Anaconda Prompt/Terminal de Anaconda)

# Leer archivo FASTA sin ayuda de librerías externas como BioPython

In [3]:
secuencias = [] # Crea una lista vacía

with open("data/adn.fasta", "r") as f: # Usa función open en modo de lectura ("r" de "read") y darle alias f
    linea = f.readline()                # Lee primera línea del archivo y la guarda en la variable nombre
    while linea != None and linea != "": # Mientras haya líneas no vacías...
        nombre = linea
        nombre = nombre.strip()          # Elimina caracteres en blanco (espacios, tabulaciones, saltos de línea) antes y después del texto
        nombre = nombre[1:]              # Elimina el carácter ">" del nombre. Toma la cadena nombre desde el carácter en el índice 1
        secuencia = f.readline().strip() # Lee segunda línea del archivo, elimina caracteres en blanco y la guarda en la variable nombre
        secuencia = secuencia.upper()    # Pone toda la secuencia en mayúsculas
        entrada = (nombre, secuencia)    # Crea una tupla con dos datos: el nombre y la secuencia de la proteína
        secuencias.append(entrada)       # Agrega la tupla a una lista
        linea = f.readline()            # Lee el nombre en la siguiente línea. Puede que no exista, lo que haría que el ciclo se frene.

# print(secuencias) # Las listas se muestran entre [], las tuplas entre ()

# Leer archivo FASTA con BioPython

In [4]:
# Crea función leerArchivo para leer archivos de secuencias de ADN o proteínas. Por defecto es de tipo fasta.
def leerArchivo(ruta, tipo="fasta"):
    entradas = [] # Crea una lista vacía

    # Usando la función parse de la clase SeqIO, lee e interpreta el archivo como un FASTA. Recorre todas las entradas una por una.
    for record in SeqIO.parse(ruta, tipo): 
        nombre = record.id              # Obtiene el nombre de la secuencia de la variable record
        secuencia = str(record.seq)     # Obtiene la secuencia de la variable record y la transforma en cadena de texto   
        secuencia = secuencia.upper()   # Pone toda la secuencia en mayúsculas
        entrada = (nombre, secuencia)   # Crea una tupla con dos datos: el nombre y la secuencia de la proteína
        entradas.append(entrada)        # Agrega la tupla a una lista

    return entradas # Las listas se muestran entre [], las tuplas entre ()
    
# entradas = leerArchivo("data/adn.fasta")

# print(entradas)

# Leer matriz BLOSUM (solo para proteínas)

In [5]:
blosum = {} # Crear matriz como un diccionario vacío
        
alfabeto = ["A", "R", "N", "D", "C", "E", "Q", "G", "H", "I", "L", "K", "M", "F", "P", "S", "T", "W", "Y", "V"]

with open("data/blosum62.txt", "r") as f: # Abrir archivo blosum62.txt en modo lectura. Este archivo está en el mismo orden que nuestro alfabeto.
    f.readline()                          # Se omite la primera línea del archivo, ya que son encabezados de columnas
    for letra in alfabeto:                # Recorre todas las letras del alfabeto en orden
        blosum[letra] = {}                # Para cada letra se asigna un nuevo diccionario
        linea = f.readline().strip()      # Se lee una línea por cada letra
        valores = linea.split()           # Se divide la línea cada vez que encuentre un espacio.
        for i in range(len(alfabeto)):     # Se recorre el listado de valores
            columna = alfabeto[i]
            blosum[letra][columna] = int(valores[i]) # Se asigna el valor correspondiente a cada letra en la fila

# print("\nMATRIZ BLOSUM62")           # \n es el carácter de salto de línea
# encabezado = "AA\t"                  # \t es el carácter de tabulación
# for letra in alfabeto:
#     encabezado += letra + "\t"
# print(encabezado)              
# for letra1 in alfabeto:
#     print(letra1 + "\t", end="") # El parámetro end="" es para que no salte a la siguiente línea
#     for letra2 in alfabeto:
#         print(str(blosum[letra1][letra2]) + "\t", end="")
#     print("")

# Imprimir tabla

In [6]:
def imprimirTabla(tabla, decimales=4):
    tablaMayusculas = tabla.upper()
    if tabla == "conteo":
        datos = conteo
    elif tabla == "frecuencias":
        datos = frecuencias
    else:
        datos = pesos
    print("\nTABLA DE " + tablaMayusculas) # \n es el carácter de salto de línea
    encabezado = "\t"                      # \t es el carácter de tabulación
    for letra in alfabeto:
        encabezado += letra + "\t"
    print(encabezado)              
    for posicion in range(len(datos)):
        print(str(posicion + 1) + " A\t", end="") # El parámetro end="" es para que no salte a la siguiente línea
        for letra in alfabeto:
            formato = "{0:." + str(decimales) + "f}"
            print(formato.format(datos[posicion][letra]) + "\t", end="")
        print("")

# Crear alfabeto para ADN o proteínas

In [7]:
def obtenerAlfabeto(entradas): # Crea función llamada obtenerAlfabeto que recibe una lista de entradas
    alfabeto = set()             # Crea un set vacío. Los sets son listas que no permiten elementos repetidos.
    
    for entrada in entradas:     # Recorre las entradas una por una
        for letra in entrada[1]: # Recorre cada letra en la secuencia de la entrada
            alfabeto.add(letra)
            
    print(alfabeto)              # Los sets se muestran entre {} 
            
    adn = ["A", "C", "T", "G"]   # Crea una lista inicializado con las letras de ADN
    
    if alfabeto.issubset(adn):   # Si el alfabeto creado a partir de las entradas solo contiene letras de ADN, es ADN
        alfabeto = adn
    else:
        alfabeto = ["A", "R", "N", "D", "C", "E", "Q", "G", "H", "I", "L", "K", "M", "F", "P", "S", "T", "W", "Y", "V"]
            
    return alfabeto

# alfabeto = obtenerAlfabeto(entradas)

# print(alfabeto)

# Calcular conteos

In [8]:
# Crea función llamada calcularConteo que recibe una lista de entradas
def calcularConteo(entradas):
    modeloConteo = {}            # Crea un diccionario vacío para el modelo de conteo
    for letra in alfabeto:       # Recorre las letras del alfabeto
        modeloConteo[letra] = 0  # Crea la llave de la letra en el diccionario en valor 0
        
    conteo = []                             # Crea una lista vacía para los conteos
    longitud = len(entradas[0][1])          # Obtiene la longitud de las secuencias
    for i in range(longitud):               # Repite el ciclo N veces, siendo N la longitud de las secuencias
        conteo.append(modeloConteo.copy())  # Copia el modelo de conteo y agrega a la lista tantos modelos como carácteres haya en las secuencias  
        
    for entrada in entradas:       # Recorre las entradas una por una
        i = 0                      # Posición de la letra actual
        for letra in entrada[1]:   # Recorre cada letra en la secuencia de la entrada
            conteo[i][letra] += 1  # Suma 1 al conteo de la letra en la posición i. También se podría con conteo[i][letra] = conteo[i][letra] + 1
            i += 1                 # Se mueve a la siguiente posición
            
    return conteo

# conteo = calcularConteo(entradas) # Guardar el conteo en la variable conteo   

# print(conteo) # Los diccionarios también se muestran entre {}, pero con el formato {llave: valor, llave: valor, ...}

# imprimirTabla("conteo", decimales=0)

# Calcular frecuencias

In [9]:
# Crea función llamada calcularFrecuencias que recibe una lista de entradas y una tabla de conteos
# ponderarSecuencia define si se calcula la frecuencia con ponderación con pseudoconteos (para evitar ceros) o no. Por defecto está activado.
# beta define el peso del pseudoconteo. Por defecto toma valor 1.
def calcularFrecuencias(entradas, conteo, ponderarSecuencia = True, beta = 1):
    modeloFrecuencias = {}            # Crea un diccionario vacío para el modelo de frecuencias
    for letra in alfabeto:            # Recorre las letras del alfabeto
        modeloFrecuencias[letra] = 0  # Crea la llave de la letra en el diccionario en valor 0
        
    frecuencias = []                        # Crea una lista vacía para las frecuencias
    longitud = len(entradas[0][1])          # Obtiene la longitud de las secuencias
    for i in range(longitud):               # Repite el ciclo N veces, siendo N la longitud de las secuencias
        frecuencias.append(modeloFrecuencias.copy())  # Copia el modelo de frecuencias y agrega a la lista tantos modelos como carácteres haya en las secuencias  
    
    if ponderarSecuencia:                      # Si se calcula frecuencia con ponderación...
        # Calcula la frecuencia previa general dependiendo de cuántas letras hay en el alfabeto.
        frecuenciasPrevias = {}
        for letra in alfabeto:
            frecuenciasPrevias[letra] = 1 / len(alfabeto)   # En este caso, todas las letras tendrán igual frecuencia previa.
        for i in range(len(conteo)):                  # Recorre la lista de conteos
            sumaPseudofrecuencias = 0                 # Variable para almacenar suma de pseudofrecuencias antes de normalizar
            for llave, valor in conteo[i].items():    # Recorre el diccionario del conteo de la posición i
                frecuencias[i][llave] = (valor + beta * frecuenciasPrevias[llave]) / len(entradas) # (Cij + b * fij) / N, N = cantidad de entradas
                sumaPseudofrecuencias += frecuencias[i][llave]  # Suma el resultado a la suma de pseudofrecuencias
            for llave in conteo[i]:
                frecuencias[i][llave] /= sumaPseudofrecuencias # Normaliza con respecto a la suma de pseudofrecuencias
    else:
        for i in range(len(conteo)):                  # Recorre la lista de conteos
            for llave, valor in conteo[i].items():    # Recorre el diccionario del conteo de la posición i
                frecuencias[i][llave] = valor / len(entradas) # Cij / N, N = cantidad de entradas
    
    return frecuencias
            
# frecuencias = calcularFrecuencias(entradas, conteo, ponderarSecuencia=True, beta=50) # Guardar el conteo en la variable frecuencias 

# print(frecuencias)

# imprimirTabla("frecuencias", decimales=3)

# Calcular pesos

In [16]:
# Crea función llamada calcularPesos que recibe una lista de entradas y una tabla de frecuencias
def calcularPesos(entradas, frecuencias):
    modeloPesos = {}                  # Crea un diccionario vacío para el modelo de pesos
    for letra in alfabeto:            # Recorre las letras del alfabeto
        modeloPesos[letra] = 0        # Crea la llave de la letra en el diccionario en valor 0
        
    pesos = []                              # Crea una lista vacía para los pesos
    longitud = len(entradas[0][1])          # Obtiene la longitud de las secuencias
    for i in range(longitud):               # Repite el ciclo N veces, siendo N la longitud de las secuencias
        pesos.append(modeloPesos.copy())    # Copia el modelo de frecuencias y agrega a la lista tantos modelos como carácteres haya en las secuencias  
    
    if len(entradas) == 1 and len(alfabeto) == 20: # Si solo hay una secuencia y se leyeron proteínas, el peso es igual al valor de la matriz BLOSUM62
        i = 0
        for letra1 in entradas[0][1]:               # Recorre cada letra de la secuencia única
            for letra2 in alfabeto:
                pesos[i][letra2] = blosum[letra1][letra2]
            i += 1
    
    else:
        # Calcula la frecuencia previa general dependiendo de cuántas letras hay en el alfabeto.
        frecuenciasPrevias = {}
        for letra in alfabeto:
            frecuenciasPrevias[letra] = 1 / len(alfabeto)   # En este caso, todas las letras tendrán igual frecuencia previa.
        for i in range(len(pesos)):                  # Recorre la lista de frecuencias
            for llave, valor in pesos[i].items():    # Recorre el diccionario del conteo de la posición i
                # CORRIGE AQUÍ - El código lanza error si no hay ponderación. Haz que esto no suceda.
                pesos[i][llave] = math.log(frecuencias[i][llave] / frecuenciasPrevias[llave])
    
    return pesos

# pesos = calcularPesos(entradas, frecuencias)

# imprimirTabla("pesos")

# Prueba con una secuencia de aminoácidos

In [11]:
entradas = leerArchivo("data/adn.fasta")
alfabeto = obtenerAlfabeto(entradas)
conteos = calcularConteo(entradas)
frecuencias = calcularFrecuencias(entradas, conteos)
pesos = calcularPesos(entradas, frecuencias)
imprimirTabla("pesos")

{'A', 'C', 'G', 'T'}

TABLA DE PESOS
	A	C	T	G	
1 A	0.9163	-0.6931	-0.6931	-0.6931	
2 A	-0.6931	0.9163	-0.6931	-0.6931	
3 A	0.9163	-0.6931	-0.6931	-0.6931	
4 A	-0.6931	0.9163	-0.6931	-0.6931	
5 A	-0.6931	-0.6931	0.9163	-0.6931	
6 A	-0.6931	-0.6931	0.9163	-0.6931	
7 A	0.9163	-0.6931	-0.6931	-0.6931	
8 A	-0.6931	0.9163	-0.6931	-0.6931	
9 A	-0.6931	-0.6931	0.9163	-0.6931	
10 A	-0.6931	-0.6931	-0.6931	0.9163	


# Prueba con una secuencia de aminoácidos

In [21]:
entradas = leerArchivo("data/proteinas-5.fasta")
alfabeto = obtenerAlfabeto(entradas)
conteos = calcularConteo(entradas)
frecuencias = calcularFrecuencias(entradas, conteos)
pesos = calcularPesos(entradas, frecuencias)
imprimirTabla("pesos")

{'A', 'P', 'C', 'G', 'V', 'R', 'Y', 'I', 'D', 'K', 'N', 'W', 'E', 'M', 'T', 'S', 'H', 'F'}

TABLA DE PESOS
	A	R	N	D	C	E	Q	G	H	I	L	K	M	F	P	S	T	W	Y	V	
1 A	-1.9459	1.0986	-1.9459	-1.9459	-1.9459	-1.9459	-1.9459	1.0986	-1.9459	-1.9459	-1.9459	1.0986	-1.9459	-1.9459	-1.9459	-1.9459	-1.9459	-1.9459	1.0986	1.7677	
2 A	1.0986	-1.9459	-1.9459	1.0986	-1.9459	-1.9459	-1.9459	1.0986	-1.9459	-1.9459	-1.9459	1.0986	1.7677	-1.9459	-1.9459	-1.9459	-1.9459	-1.9459	-1.9459	-1.9459	
3 A	-1.9459	1.0986	-1.9459	-1.9459	1.0986	-1.9459	-1.9459	-1.9459	-1.9459	-1.9459	-1.9459	-1.9459	-1.9459	-1.9459	1.0986	1.0986	1.0986	1.0986	-1.9459	-1.9459	
4 A	-1.9459	1.0986	1.0986	1.0986	1.0986	-1.9459	-1.9459	-1.9459	-1.9459	-1.9459	-1.9459	-1.9459	-1.9459	-1.9459	-1.9459	1.7677	-1.9459	-1.9459	-1.9459	-1.9459	
5 A	1.0986	1.0986	1.0986	-1.9459	-1.9459	1.0986	-1.9459	-1.9459	-1.9459	-1.9459	-1.9459	1.7677	-1.9459	-1.9459	-1.9459	-1.9459	-1.9459	-1.9459	-1.9459	-1.9459	
6 A	-1.9459	1.0986	1.0986	-1.9459	-1.9459	1.0986	-1.

# Cálculo de eficiencia de matriz PSSM

### Cargar datos

In [13]:
from scipy.stats import pearsonr # Importar función para calcular PCC

secuencias = []              # Lista vacía para guardar las secuencias
puntajesOriginales = []      # Lista vacía para guardar los puntajes del archivo

with open("data/test1.lig", "r") as f: # Leer archivo de entrada
    linea = f.readline()                   # Lee la primera línea del archivo
    while linea != None and linea != "":   # Mientras haya líneas no vacías...
        datos = linea.split()              # Separa la línea por espacio
        secuencia = datos[0]               # El primer elemento es la secuencia
        puntaje = float(datos[1])          # El segundo elemento es el valor original. Se transforma en float.
        secuencias.append(secuencia)       # Agrega la secuencia a la lista de secuencias
        puntajesOriginales.append(puntaje) # Agrega el puntaje a la lista de puntajes originales
        linea = f.readline()               # Lee la siguiente línea

print(secuencias)
print(puntajesOriginales)

['ILYQVPFSV', 'VVMGTLVAL', 'ILDEAYVMA', 'KILSVFFLA', 'HLYQGCQVV']
[0.8532, 0.5891, 0.4941, 0.8512, 0.5386]


### Función para calcular puntajes

In [14]:
# Crea función llamada calcularPuntajes que recibe una lista de entradas y una tabla PSSM
def calcularPuntajes(secuencias, pssm):
    puntajes = []                  # Crea una lista vacía para los puntajes calculados
    longitud = len(secuencias[0])  # Longitud de las secuencias (se presupone que todas tienen la misma longitud)
    for secuencia in secuencias:   # Recorre una por una las secuencias
        puntaje = 0                # Variable para guardar el puntaje calculado de la secuencia actual
        for i in range(longitud):  # Recorre letra por letra de la secuencia
            letra = secuencia[i]
            puntaje += pesos[i][letra] # Suma el peso de la matriz para la letra en la posición actual al puntaje total
        puntajes.append(puntaje)   # Agrega el puntaje calculado a la lista de puntajes
    return puntajes

puntajesCalculados = calcularPuntajes(secuencias, pesos)

print(puntajesCalculados)

[-11, -10, -6, -17, -13]


### Cálculo de coeficiente

In [15]:
pearsonr(puntajesOriginales, puntajesCalculados) # El primer valor es el coeficiente. Se puede ignorar el segundo.

(-0.6387375632182636, 0.24604048827429162)